<a href="https://colab.research.google.com/github/pollinations/hive/blob/main/notebooks/7%20Video-To-Video/3%20FiLM%20-%20Frame%20Interpolation%20for%20Large%20Motion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![preview](https://i.imgur.com/NliA9Kj.gif)

Get really cool results by adding realistic movement between images. Mix different images for surprising results.
---

Credits:[DigThatData](https://twitter.com/DigThatData)
License: MIT

In [ ]:

# Upload a zip file containing images or a video file
image_file = ''  #@param {type: "string", accepted_files: "image,video,application"}

# If you start with $n$ frames in `frames_dir` and set `recursive_interpolation_passes` to $k$, your total number of frames after interpolation will be: 2^k (n-1) -1
recursive_interpolation_passes = 1 #@param {type: "number"}

# Frame rate of output video
output_video_fps = 30 #@param {type: "number"}

#@markdown ---

output_path = "/content"

In [ ]:
!rm -r /content/frames
!mkdir -p /content/frames
if image_file.endswith('.zip'):
  !unzip -j -o "{image_file}" -d /content/frames
  !rm /content/frames/.*.*
else:
  !ffmpeg -i "{image_file}" /content/frames/frame%5d.jpg

In [ ]:

# Load a folder of images and crop and resize them to the resolution of the smallest images

import os
import cv2
import numpy as np
from glob import glob

def crop_resize_to_smallest_image(folder_path):
    """
    Load a folder of images and crop and resize them to the resolution of the smallest images
    :param folder_path: path to the folder containing images
    :param output_folder_path: path to the folder where the cropped and resized images will be saved
    :return:
    """
    # Get the list of images in the folder
    images = os.listdir(folder_path)

    # Get the size of the smallest image
    smallest_image_size = get_smallest_image_size(folder_path)

    print(smallest_image_size)
    # Iterate over the images
    for image in images:
        # Load the image
        image_path = os.path.join(folder_path, image)
        try:
          image_array = cv2.imread(image_path)

          # Crop and resize the image
          image_array = crop_and_resize_image(image_array, smallest_image_size)

      
          cv2.imwrite(image_path, image_array)
        except:
          print("image was not able to be resized. maybe no image", image_path)
    

def get_smallest_image_size(folder_path):
    """
    Get the size of the smallest image in the folder
    :param folder_path: path to the folder containing images
    :return: the size of the smallest image
    """
    # Get the list of images in the folder
    images = glob(folder_path+"/*.jpg") + glob(folder_path+"/*.png") + glob(folder_path+"/*.jpeg")

    # Get the size of the smallest image
    smallest_image_size = None
    for image in images:
        # Load the image
        image_path = os.path.join(folder_path, image)
        image_array = cv2.imread(image_path)
        # Get the size of the image
        image_size = image_array.shape

        # Check if the size of the image is smaller than the smallest image size
        if smallest_image_size is None or image_size[0] < smallest_image_size[0] or image_size[1] < smallest_image_size[1]:
            smallest_image_size = image_size

    return smallest_image_size


def crop_and_resize_image(image_array, smallest_image_size):
    """
    Crop and resize the image
    :param image_array: the image array
    :param smallest_image_size: the size of the smallest image
    :return: the cropped and resized image array
    """

    # Resize the image
    image_array = cv2.resize(image_array, (smallest_image_size[1],smallest_image_size[0]))

    return image_array

crop_resize_to_smallest_image('/content/frames')

In [ ]:
# This top bit only neneds to be run once
# ...may as well set up the notebook to 
# assume the user is only going to run all
# of setup once.
from pathlib import Path
import os

drive_mounted = False
gdrive_fpath = '.'
local_path = '/content/'

##############################

#@markdown # 1. Setup Workspace

#@markdown Run this cell to perform setup.

#@markdown Mounting your google drive is optional.
#@markdown If you mount your drive, code and models will be downloaded to it.
#@markdown This should reduce setup time after your first run.

###################

# Optionally Mount GDrive

mount_gdrive = False # @param{type:"boolean"}
if mount_gdrive and not drive_mounted:
    from google.colab import drive

    gdrive_mountpoint = '/content/drive/' #@param{type:"string"}
    gdrive_subdirectory = 'MyDrive/interpolation' #@param{type:"string"}
    gdrive_fpath = str(Path(gdrive_mountpoint) / gdrive_subdirectory)
    try:
        drive.mount(gdrive_mountpoint, force_remount = True)
        !mkdir -p {gdrive_fpath}
        %cd {gdrive_fpath}
        local_path = gdrive_fpath
        drive_mounted = True
    except OSError:
        print(
            "\n\n-----[PYTTI-TOOLS]-------\n\n"
            "If you received a scary OSError and your drive"
            " was already mounted, ignore it."
            "\n\n-----[PYTTI-TOOLS]-------\n\n"
            )
        raise

#####################

# Perform rest of setup

if not Path('./frame-interpolation').exists():
    !git clone https://github.com/pytti-tools/frame-interpolation

try:
    import frame_interpolation
except ModuleNotFoundError:
    %pip install -r ./frame-interpolation/requirements_colab.txt
    %pip install ./frame-interpolation

#url = "https://drive.google.com/drive/folders/1GhVNBPq20X7eaMsesydQ774CgGcDGkc6?usp=sharing"
share_id = "1GhVNBPq20X7eaMsesydQ774CgGcDGkc6"

if not (Path(local_path) / 'saved_model').exists():
    !pip install --upgrade gdown
    !gdown --folder {share_id}

# create default frame
!mkdir -p frames

In [ ]:
#@markdown # 2. Setup Interpolation

#@markdown Specify the directory containing your video frames with the `frames_dir` parameter.

frames_dir = "/content/frames" #@param{'type':'string'}

#@markdown A single pass of the interpolation procedure adds a frame between each contiguous pair of frames in `frames_dir`.


#@markdown Check this box to generate a video output. If no output video will be generated, the FPS option can be ignored.
output_video = True #@param{'type':'boolean'}


In [ ]:
#@markdown # 3. Interpolate!
#@markdown Results will be saved in a subdirectory of `frames_dir` named "interpolated_frames"

from loguru import logger

logger.info("Beginning interpolation...")

if output_video:
  !python -m frame_interpolation.eval.interpolator_cli \
      --model_path ./saved_model \
      --pattern {frames_dir} \
      --fps {output_video_fps} \
      --output_video
      

else:
    !python -m frame_interpolation.eval.interpolator_cli \
      --model_path ./saved_model \
      --pattern {frames_dir} \

logger.info("Interpolation comlpete.")

In [ ]:
!cp -v /content/frames/interpolated.mp4 {output_path}
import os.path
if not os.path.exists(output_path + '/interpolated.mp4'):
  raise Exception("Expected output file does not exist.")

```
MIT License

Copyright (c) 2022 David Marx

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```